Notebook ini menyajikan pengolahan nilai setiap mata kuliah / tingkat / prodi pada dua Semester Tahun Ajaran yang diberikan pengguna sebagai masukan. 

Hasil pengolahan adalah tabel yang menyajikan rata-rata Indeks Prestasi Semester pada dua Semester Tahun Ajaran yang diberikan.

In [ ]:
import pandas as pd

In [ ]:
# simpan informasi stata dan nama prodi dalam sebuah tupel
tupl_prodi = tuple([['D3', 'Teknologi Informasi'],
                    ['D3', 'Teknologi Komputer'],
                    ['D4', 'Teknologi Rekayasa Perangkat Lunak'],
                    ['D4', 'Teknologi Rekayasa Perangkat Lunak (Tolikara)'],
                    ['S1', 'Informatika'],
                    ['S1', 'Sistem Informasi'],
                    ['S1', 'Teknik Elektro'],
                    ['S1', 'Teknik Bioproses'],
                    ['S1', 'Manajemen Rekayasa']])

# simpan informasi grade dalam sebuah dictionary
dict_grade = {'A' : 4.0,
              'AB' : 3.5,
              'B' : 3.0,
              'BC' : 2.5,
              'C' : 2.0,
              'D' : 1.0,
              'E' : 0,
              'T' : float('NaN')}

In [ ]:
def memuat_data_ke_df(prodi, angkatan):
    angkatan = str(angkatan)
    if prodi == 'Teknologi Rekayasa Perangkat Lunak (Tolikara)':
        df = pd.read_excel('./nilai/Rekap_Nilai_' + 
                           prodi.replace(' (Tolikara)','') + 
                           '_' + angkatan + '.xlsx',
                       header = [0, 1, 2, 3])
    else:
        df = pd.read_excel('./nilai/Rekap_Nilai_' + 
                           prodi + '_' + 
                           angkatan + '.xlsx',
                       header = [0, 1, 2, 3])
    return df

In [ ]:
def prapemrosesan_data(df, semester, prodi, angkatan):
    semester = str(semester)
    angkatan = str(angkatan)
    
    df_grade = df.xs('Semester ' + semester, level=0, axis=1)
    df_grade = df_grade.iloc[: , :-4]
    
    cols_km = [str(c) for c in list(df_grade.columns.get_level_values(1))[:] 
              if c != 'Grade' and not 'Unnamed' in str(c)]
    cols_km.insert(0, "Nim")
    
    cols = [c for c in df_grade.columns if 'Grade' in c]
    df_grade = df_grade[cols]
    
    dft = pd.concat([df.xs('Nim', level=3, axis=1),
                     df_grade], axis=1)
    
    lst_tmp = []
    for idx, col in enumerate(dft):
        if idx == 0:
            lst_tmp.append(('SKS', 'Grade', cols_km[idx]))
        else:
            lst_tmp.append((col[0], col[1], cols_km[idx]))
    
    # sesuaikan nama kolom
    dft.columns = pd.MultiIndex.from_tuples(tuple(lst_tmp))
    
    # substitusi semua grade menjadi nilai skala 0-4
    for i in dft:
        dft[i] = [dict_grade.get(item,item) for item in dft[i].tolist()]
        
    # penanganan khusus Prodi Teknologi Rekayasa Perangkat Lunak (Tolikara)
    if angkatan == str('2021'):
        if prodi == 'Teknologi Rekayasa Perangkat Lunak (Tolikara)':
            dft = dft[:25]
            dft = dft.dropna(axis=1)
        elif prodi == 'Teknologi Rekayasa Perangkat Lunak':
            dft = dft[25:]
    
    # hapus baris data apabila terdapat 50% nilai 0 
    mask = (dft.iloc[:, 1:] == float(0)).sum(1) >= len(dft.columns[1:]) / 2
    dft = dft[~mask]
    
    # hapus setiap kolom yang semua nilainya NaN
    dft = dft.dropna(how='all', axis=1)
    
    # hapus setiap baris yang semua nilainya NaN
    dft = dft.dropna(how='all', subset=dft.columns[1:])
    
    # reset index
    dft = dft.reset_index(drop=True)
    
    return dft

In [ ]:
# fungsi untuk menghitung indeks prestasi semester berdasarkan
# list grade dan list bobot sks
def kalkulasi_ips(lst_grd, lst_sks):
    # pastikan list sks tidak mengandung NaN
    # NaN berarti mahasiswa tersebut tidak mengambil
    # mata kuliah yang dimaksud
    temp = [i for i, e in enumerate(lst_grd) if str(e).lower() == 'nan']
    
    lst_grd = [element for (i,element) in enumerate(lst_grd) if i not in temp]
    lst_sks = [element for (i,element) in enumerate(lst_sks) if i not in temp]
    
    a = sum(x * y for x, y in zip(lst_grd, lst_sks))
    b = sum(lst_sks)
    
    return round(a / b, 2)

In [ ]:
# fungsi untuk menghitung nilai rata-rata elemen sebuah list
def kalkulasi_rerata(list_masukan):
    # nilai rata-rata dibulatkan sampai 2 desimal di
    # belakang koma
    return round(sum(list_masukan) / len(list_masukan), 2)

In [ ]:
def kalkulasi_rerata_ips(df):
    # list sks
    lst_sks = list(df.columns.get_level_values(0))[1:]

    # list ips
    lst_tmp =[]
    for i in df.itertuples():
        lst_grd = list(i[2:])
        kalkulasi_ips(lst_grd, lst_sks)
        lst_tmp.append(kalkulasi_ips(lst_grd, lst_sks))
    
    return kalkulasi_rerata(lst_tmp)

In [ ]:
def generate_info(tahun_ajaran):
    aw, ak = tahun_ajaran[:len(tahun_ajaran)//2], tahun_ajaran[len(tahun_ajaran)//2:]
    aw = int("20" + aw)
    ak = int("20" + ak)
    
    dict_info = {}
    tingkat = 4
    semester = 8
    for i in range(aw-3, aw+1): 
        dict_info.update({str(tingkat) : {'Angkatan' : i ,
                          'Semester' : semester}})
        tingkat -= 1
        semester -= 2
    
    df_hasil = pd.DataFrame(columns=['Strata',
                                     'Prodi',
                                     'Tingkat',
                                     'Angkatan',
                                     'Semester'])
    
    for idx, ele in enumerate(tupl_prodi):
        srata = ele[0]
        for key in dict_info:
            to_append = [srata, ele[1], key,
                         dict_info[key]['Angkatan'], dict_info[key]['Semester']]
            a_series = pd.Series(to_append, index = df_hasil.columns)
            df_hasil = df_hasil.append(a_series, ignore_index=True)
    
    df_hasil.drop(df_hasil[(df_hasil['Strata'] == 'D3')  & (df_hasil['Tingkat'] == '4')].index, inplace = True)
    df_hasil.drop(df_hasil[(df_hasil['Prodi'] == 'Teknologi Rekayasa Perangkat Lunak (Tolikara)')  & 
                           ((df_hasil['Tingkat'] == '4') |
                            (df_hasil['Tingkat'] == '3') |
                            (df_hasil['Tingkat'] == '2')
                           )].index, inplace = True)
    
    return df_hasil

In [ ]:
def proses(semester, prodi, angkatan):
    s = str(semester)
    p = str(prodi)
    a = str(angkatan)

    df_mentah = memuat_data_ke_df(p, a)
    df = prapemrosesan_data(df_mentah, s, p, a)
    
    return df

In [ ]:
def membandingkan_tahun_ajaran(x, y):
    df_x = generate_info(x)
    df_y = generate_info(y)
    
    df_xy = pd.merge(df_x, df_y, on=['Strata', 'Prodi', 'Tingkat', 'Semester'])
    
    df_hasil = pd.DataFrame(columns=['Prodi',
                                     'Tingkat',
                                     'Rata-rata ' + str(x[:2] + '/' + x[2:]),
                                     'Rata-rata ' + str(y[:2] + '/' + y[2:]),
                                     'Kode MK'])
    
    for i in df_xy.itertuples():
        df_ix = proses(i.Semester, i.Prodi, i.Angkatan_x)
        df_iy = proses(i.Semester, i.Prodi, i.Angkatan_y)

        # interseksi kolom
        cols = df_ix.columns & df_iy.columns
        mk = cols.get_level_values(2).tolist()[1:]
        
        if len(mk) == 0:
            a = float('NaN')
            b = float('NaN')
        else:
            # kalkulasi rata-rata berdasarkan interseksi kolom
            try:
                a = kalkulasi_rerata_ips(df_ix)
            except Exception as e:
                print('Pada', i.Semester, i.Prodi, i.Angkatan_x, e)
                a = float('NaN')

            try:
                b = kalkulasi_rerata_ips(df_iy)
            except Exception as e:
                print('Pada', i.Semester, i.Prodi, i.Angkatan_y, e)
                b = float('NaN')

        to_append = [i.Prodi,
                     i.Tingkat,
                     a, b, mk]
        a_series = pd.Series(to_append, index = df_hasil.columns)
        df_hasil = df_hasil.append(a_series, ignore_index=True)
            
    return df_hasil

In [ ]:
display(membandingkan_tahun_ajaran('1920', '2021'))

**Copyright © 2021 Biro Administrasi dan Akademik [Institut Teknologi Del](https://www.del.ac.id/). All Rights Reserved**

---